In [11]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer


import matplotlib.pyplot as plt
import pickle
from preprocess import make_one_hot_encoding

In [2]:
def train_model(train_df, train_target):
    model = RandomForestRegressor( n_estimators=100, n_jobs=-1, random_state=42 ).fit(train_df, train_target)
    print("training is complete")
    return model

In [3]:
def save_model(model, dv):
    with open('model.pkl', 'wb') as f_out:
        pickle.dump((dv,model), f_out)

In [19]:
def make_label_encoding(df):
    
    encoder = LabelEncoder()
    df["Gender"] = encoder.fit_transform(df["Gender"])
    dv = DictVectorizer()

    dicts = df.to_dict(orient="records")
    dicts = dv.fit_transform(dicts)
    return df, dv

In [4]:
import mlflow

In [5]:
import mlflow


mlflow.set_tracking_uri("sqlite:///final_project.db")
mlflow.set_experiment("car-prediction-experiment")

2022/08/03 09:02:13 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/08/03 09:02:13 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='car-prediction-experiment', tags={}>

In [20]:
df = pd.read_csv(r"../data/car_data.csv")
df, dv = make_label_encoding(df)
X = df[["Gender", "Age", "AnnualSalary"]]
y = df["Purchased"]

In [21]:
train_df, test_df, train_target, test_target = train_test_split(X, y, test_size = 0.33)

In [29]:
dv = DictVectorizer()

In [30]:
train_dicts = train_df.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

test_dicts = test_df.to_dict(orient='records')
X_test = dv.transform(test_dicts)

In [31]:
model = train_model(X_train, train_target)

training is complete


In [32]:
with open('rf.pkl', 'wb') as f_out:
    pickle.dump((dv, model), f_out)

In [36]:
dv.get_feature_names_out()

array(['Age', 'AnnualSalary', 'Gender'], dtype=object)

In [40]:
train_df

,Gender,Age,AnnualSalary
784,1,35,44500
464,0,38,63500
188,0,45,110500
333,1,28,89000
222,0,47,75500
...,...,...,...
709,0,42,61500
959,0,48,50500
189,0,55,40500
373,0,48,48500
